# GraphGuard

***Locate and find Classes in Apks with updated Obfuscation Mapping***


Processing Steps:
1. String Matcher (Finding Classes and Methods
  * Counting Strings used in Classes and Methods and try to find exact matching counter.
  * Find Classes by identifying Strings used only in this single Class.
2. Structure Matcher (Finding Classes)
  * Modifiers of class
  * Modifiers, Parameters, Parameter Types and Return Types of Methods
  * Number and Types of Fields.
3. Method Matcher (Find Methods from matching Classes)
  * Modifiers
  * Return Type, Parameter Types
  * Bytecode Length
  * References to and from


***If you use Akrolyb, please have a look at [Akrolyb Interoptability](#Interoptability-with-Akrolyb)***

In [37]:
%matplotlib notebook

%load_ext autoreload
%autoreload

from IPython.core.display import display, HTML
display(HTML("<style>div.output_area pre {white-space: pre;}</style>"))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [38]:
from core.start import process_files
from utils.formats import *

from core.accumulator import Accumulator
from core.decs import *

from strategies import\
    strings as strings_strategy,\
    methods as methods_strategy,\
    structures as structures_strategy,\
    fields as fields_strategy

from core.strategy_handler import StrategyHandler, FLAG_CLASS, FLAG_METHOD, FLAG_FIELD

from utils import generate

# Loading Androguard

The following code loads the files and starts Androguard

It should support multiprocessing, however the Pipe communication seems to break when transmitting the processed Androguard Objects. I suspect the Object is simply too big for Pickle to serialize or another component in the transmitting chain.

In [39]:
AG_SESSION_FILE = "./Androguard.ag"
MULTIPROCESS_FILES = False  # Currently not working due to serialization issues


# Strategy Rules & Parameters
strings_strategy.MAX_USAGE_COUNT_STR = 20
strings_strategy.UNIQUE_STRINGS_MAJORITY = 2 / 3

methods_strategy.MIN_MATCH_POINTS = 2


# APK Files to load
file_paths = (
    "../../../Downloads/com.snapchat.android_10.86.5.61-2069_minAPI19(arm64-v8a)(nodpi)_apkmirror.com.apk",
    "../../../Downloads/com.snapchat.android_10.87.5.69-2074_minAPI19(arm64-v8a)(nodpi)_apkmirror.com.apk"
)
file_paths = (
#     "../../../Downloads/com.instagram.android_143.0.0.25.121-216817305_minAPI23(arm64-v8a)(nodpi)_apkmirror.com.apk",
    
    "../../../Downloads/com.instagram.android_151.0.0.23.120-232868015_minAPI23(arm64-v8a)(nodpi)_apkmirror.com.apk",
#     "../../../Downloads/com.instagram.android_151.0.0.23.120-232867993_minAPI19(armeabi-v7a)(nodpi)_apkmirror.com.apk",
    
#     "../../../Downloads/com.instagram.android_152.0.0.25.117-234847234_minAPI23(arm64-v8a)(nodpi)_apkmirror.com.apk",
    "../../../Downloads/com.instagram.android_152.0.0.25.117-234847224_minAPI19(armeabi-v7a)(nodpi)_apkmirror.com.apk"
)

In [4]:
(a, d, dx), (a2, d2, dx2) = process_files(file_paths, MULTIPROCESS_FILES)

Loading Session from Apk at ../../../Downloads/com.instagram.android_151.0.0.23.120-232868015_minAPI23(arm64-v8a)(nodpi)_apkmirror.com.apk
Loading Session from Apk at ../../../Downloads/com.instagram.android_152.0.0.25.117-234847224_minAPI19(armeabi-v7a)(nodpi)_apkmirror.com.apk


# List of Classes, Methods and Variables

Defining Elements that GraphGuard should try to match (obviously requires full class names).
* `c_decs` in form of a tuple of Strings
* `m_decs` in form of a tuple of MethodDeclarations
* `f_decs` in form of a tuple of FieldDeclarations

In [40]:
# Classes of the Methods and Fields are automatically merged with c_decs
c_decs = (
    "com.snap.mushroom.MainActivity",
)

In [41]:
m_decs = (
    MethodDec('QE5', 'a', 'QE5', 'PE5'),
    MethodDec('V0h', 'j0', 'IMd'),
    MethodDec('vre', 'g', 'pX4'),
    MethodDec('FOb', '<init>', 'java.lang.String', 'boolean', 'java.lang.String', 'java.lang.String', 'java.lang.String', 'java.lang.Long', 'Xt5', 'long', 'boolean', 'java.lang.Long', 'long')
)

In [48]:
f_decs = tuple()

In [49]:
# Add Classes of Methods and Fields to c_decs
orig_c_decs, c_decs = c_decs, tuple(map(lambda m: m.class_name, m_decs)) \
        + tuple(map(lambda v: v.class_name, f_decs)) + c_decs

# Processing and Matching

Loading the accumulator, an object that manages all the possible candidates that are matched by the different Matchers, and extracts the matching candidates. It also performs Inner joins on previous candidates to find the exact (or optimal) match.

In [50]:
accumulator = Accumulator()

Resolving the previously defined MethodDecs. If this fails, the MethodDecs are wrong and contain an error. Make sure the method specified with the MethodDec exists.

In [52]:
dec_cas = resolve_classes(dx, c_decs)

r_cas = tuple(dec_cas.values())
r_mas = resolve_methods(m_decs, dec_cas)
r_fas = resolve_fields(f_decs, dec_cas, dx)

print("Resolved all Classes, Methods and Fields")
if False:
    print("", *map(pretty_format_ma, r_mas), sep="\n* ")

# Arguments to provide to Strategies
s_args = (dx, dx2, r_cas, r_mas, r_fas, accumulator)

Resolved all Classes, Methods and Fields


In [35]:
strategies = StrategyHandler()

## String Strategy

### Exact Counter Match

Extracts Strings used either in the given methods directly or in the classes the methods define for both, the old version and the new version. It then compares the Counters for classes and methods and tries to find exact matches between the Counter Objects.

In [36]:
def string_counter_strategy(r_cas, r_mas, r_fas):
    string_s = strings_strategy.StringStrategy(dx, dx2, r_cas, r_mas, r_fas, accumulator)
    candidates_cs, candidates_ms = string_s.compare_counters()
    
    accumulator.add_candidates(candidates_cs, candidates_ms)

strategies.add_strategy(string_counter_strategy, FLAG_CLASS | FLAG_METHOD)
strategies.invoke_strategies(r_cas, r_mas, r_fas, only_new=True)

+ Found single candidate for Method. Considering it a match 
	com.instagram.direct.visual.DirectVisualMessageViewerController#A04(com.instagram.direct.visual.DirectVisualMessageViewerController) -> com.instagram.direct.visual.DirectVisualMessageViewerController#A04(com.instagram.direct.visual.DirectVisualMessageViewerController)
+ Found single candidate for Method. Considering it a match 
	X.1cE#A01() -> X.1cC#A01()
+ Found single candidate for Method. Considering it a match 
	X.3Gu#A02(X.3Gx, android.content.Context, X.0C8, X.0hu, X.0Qp, com.instagram.profile.fragment.UserDetailDelegate, boolean, X.1Ol, java.lang.Integer, boolean, boolean, X.1a7) -> X.2sY#A02(X.2sc, android.content.Context, X.0Ez, X.0iD, X.0Ra, com.instagram.profile.fragment.UserDetailDelegate, boolean, X.1Or, java.lang.Integer, boolean, boolean, X.1a8)
+ Found single candidate for Method. Considering it a match 
	com.instagram.realtimeclient.RealtimeClientManager#sendCommand(java.lang.String, java.lang.String, com.in

### Unique Strings

Gather all Strings that are used only in a single class ("Unique Strings") that we still need to match. Then try to find the matching class by only searching for the Unique Strings.

In [13]:
def unique_strings_strategy(r_cas, r_mas, r_fas):
    string_s = strings_strategy.StringStrategy(dx, dx2, r_cas, r_mas, r_fas, accumulator)
    candidates_cs = string_s.compare_unique_strings()
    
    accumulator.add_candidates(candidates_cs)

strategies.add_strategy(unique_strings_strategy, FLAG_CLASS)
strategies.invoke_strategies(r_cas, r_mas, r_fas, only_new=True)

~ Unique String not used in single class anymore. Change! (biography_product_mentions)
+ Found single candidate for Class. Considering it a match! 
	LX/0C8; -> LX/0Ez;
+ Found single candidate for Class. Considering it a match! 
	LX/0gz; -> LX/0hD;
+ Found single candidate for Class. Considering it a match! 
	LX/0oL; -> LX/0oA;
+ Found single candidate for Class. Considering it a match! 
	LX/0r4; -> LX/0ob;
+ Found single candidate for Class. Considering it a match! 
	LX/0yv; -> LX/0zR;
+ Found single candidate for Class. Considering it a match! 
	Lcom/instagram/mainactivity/MainActivity; -> Lcom/instagram/mainactivity/MainActivity;
+ Found single candidate for Class. Considering it a match! 
	Lcom/instagram/ui/swipenavigation/SwipeNavigationContainer; -> Lcom/instagram/ui/swipenavigation/SwipeNavigationContainer;
+ Found single candidate for Class. Considering it a match! 
	LX/1Fy; -> LX/1GD;
+ Found single candidate for Class. Considering it a match! 
	LX/1J0; -> LX/1Jo;
+ Found sing

## Structure Strategy

Iterating through every single class and checks for each unmatched class if both have a similar "Profile":
* Number of Methods and Fields
* Types of Fields and Descriptors of Methods

In [14]:
def structure_strategy(r_cas, r_mas, r_fas):
    structure_s = structures_strategy.StructureStrategy(dx, dx2, r_cas, r_mas, r_fas, accumulator)
    candidates_cs = structure_s.get_exact_structure_matches()

    accumulator.add_candidates(candidates_cs)

strategies.add_strategy(structure_strategy, FLAG_CLASS)
strategies.invoke_strategies(r_cas, r_mas, r_fas, only_new=True)

+ Found single candidate for Class. Considering it a match! 
	LX/2vv; -> LX/39Y;
+ Found single candidate for Class. Considering it a match! 
	Lcom/instagram/debug/devoptions/api/DeveloperOptionsLauncher; -> Lcom/instagram/debug/devoptions/api/DeveloperOptionsLauncher;
+ Found single candidate for Class. Considering it a match! 
	LX/3k8; -> LX/3FC;
+ Found single candidate for Class. Considering it a match! 
	LX/50B; -> LX/4hE;
+ Found single candidate for Class. Considering it a match! 
	LX/0h8; -> LX/0hM;
+ Found single candidate for Class. Considering it a match! 
	LX/3KR; -> LX/34n;
+ Found single candidate for Class. Considering it a match! 
	LX/1Wj; -> LX/1Wl;
+ Found single candidate for Class. Considering it a match! 
	LX/1lQ; -> LX/1lj;
* Found multiple candidates for Class LX/20d;
* Found multiple candidates for Class LX/29m;
+ Found single candidate for Class. Considering it a match! 
	LX/1Wo; -> LX/1Wq;
+ Found single candidate for Class. Considering it a match! 
	LX/0jM; -

## Method Strategy

Uses different weighted criteria to get exact or optimal matches. The criteria are:
* Modifiers
* Return Type and Parameter Types
* Length of Bytecode
* References to and from

In [15]:
def method_strategy(r_cas, r_mas, r_fas):
    method_s = methods_strategy.MethodStrategy(dx, dx2, r_cas, r_mas, r_fas, accumulator)

    print("Exact Matching")
    print()

    # Exact Matches
    candidates_ms = method_s.try_resolve_ms(True)
    accumulator.add_candidates(candidates_ms=candidates_ms)

    print()
    print("Non-Exact Matching")
    print()

    method_s.update_matched()
    # Non-Exact Matches by using weights on the criteria
    candidates_ms = method_s.try_resolve_ms(False)
    accumulator.add_candidates(candidates_ms=candidates_ms)
    
strategies.add_strategy(method_strategy, FLAG_METHOD)
strategies.invoke_strategies(r_cas, r_mas, r_fas, only_new=True)

Exact Matching

Class not resolved for Method X.20d#onDoubleTap(android.view.MotionEvent)
Class not resolved for Method X.29m#onDoubleTap(android.view.MotionEvent)
+ Found single candidate for Method. Considering it a match 
	X.2vv#onLongPress(android.view.MotionEvent) -> X.39Y#onLongPress(android.view.MotionEvent)
* Found multiple candidates for Method com.instagram.debug.devoptions.api.DeveloperOptionsLauncher#launchStoryToolbarSwitcherTool(android.content.Context, androidx.fragment.app.FragmentActivity, X.0C8)
* Found multiple candidates for Method com.instagram.debug.devoptions.api.DeveloperOptionsLauncher#launchStoriesExperimentSwitcherTool(android.content.Context, androidx.fragment.app.FragmentActivity, X.0C8)
+ Found single candidate for Method. Considering it a match 
	com.instagram.direct.visual.DirectVisualMessageViewerController#A0C(com.instagram.direct.visual.DirectVisualMessageViewerController, java.lang.String) -> com.instagram.direct.visual.DirectVisualMessageViewerContr

## Field Strategy

Gathers all fields of matching Class and filters using the following criteria:
* By Type if a matching Type has been found
* By weighted sub-criteria such as:
  * Access Flags
  * Size
  * Number of reads & Number of writes
* By Index if the list of Fields has not changed dramatically

In [16]:
def field_strategy(r_cas, r_mas, r_fas):
    field_s = fields_strategy.FieldStrategy(*s_args)
    
    print("Resolving Types of Fields")
    strategies.invoke_strategies(tuple(field_s.get_types_to_match()))
    print()
    
    candidates_fs = field_s.try_resolve_fs()
    
    accumulator.add_candidates(candidates_fs=candidates_fs)
    
strategies.add_strategy(field_strategy, FLAG_FIELD)
strategies.invoke_strategies(r_cas, r_mas, r_fas, only_new=True)

Resolving Types of Fields
Could resolve 0 new Classes, 0 new Methods, 0 new Fields.
+ Found single candidate for Class. Considering it a match! 
	LX/0hu; -> LX/0iD;
+ Found single candidate for Class. Considering it a match! 
	Lcom/instagram/common/ui/widget/imageview/IgImageView; -> Lcom/instagram/common/ui/widget/imageview/IgImageView;
+ Found single candidate for Class. Considering it a match! 
	LX/1Rl; -> LX/1R1;
+ Found single candidate for Class. Considering it a match! 
	LX/1gl; -> LX/1wo;
Could resolve 4 new Classes, 0 new Methods, 0 new Fields.
+ Found single candidate for Class. Considering it a match! 
	LX/0rC; -> LX/0of;
+ Found single candidate for Class. Considering it a match! 
	Lcom/instagram/igtv/ui/GestureManagerFrameLayout; -> Lcom/instagram/igtv/ui/GestureManagerFrameLayout;
+ Found single candidate for Class. Considering it a match! 
	LX/0PS; -> LX/0QM;
+ Found single candidate for Class. Considering it a match! 
	LX/1x6; -> LX/24y;
+ Found single candidate for Cla

# Results
Display summary and matching pairs.

Output to compatible MethodDec input to update from auto-updated values.

In [17]:
print(len(accumulator.matching_cs), "/", len(orig_c_decs), "Classes were matched")
print(len(accumulator.matching_ms), "/", len(m_decs), "Methods were matched")
print(len(accumulator.matching_fs), "/", len(f_decs), "Fields were matched")

print()
print("Matching Classes:")
for c1, c2 in accumulator.matching_cs.items():
    print("•", pretty_format_class(c1), "->", pretty_format_class(c2))

print()
print("Matching Methods:")
for m, ma in accumulator.matching_ms.items():
    print("•", pretty_format_ma(m), "->", pretty_format_ma(ma))

print()
print("Matching Fields:")
for fa, fa2 in accumulator.matching_fs.items():
    print("•", pretty_format_fa(fa.get_field()), "->", pretty_format_fa(fa2))

58 / 1 Classes were matched
50 / 55 Methods were matched
22 / 25 Fields were matched

Matching Classes:
• com.instagram.direct.visual.DirectVisualMessageViewerController -> com.instagram.direct.visual.DirectVisualMessageViewerController
• X.1cE -> X.1cC
• X.3Gu -> X.2sY
• com.instagram.realtimeclient.RealtimeClientManager -> com.instagram.realtimeclient.RealtimeClientManager
• X.3zU -> X.3pf
• X.1Ol -> X.1Or
• X.1Z4 -> X.1Z7
• X.1x5 -> X.24x
• X.4Gh -> X.45p
• X.DJ0 -> X.Amd
• X.DJ1 -> X.Ame
• X.0C8 -> X.0Ez
• X.0gz -> X.0hD
• X.0oL -> X.0oA
• X.0r4 -> X.0ob
• X.0yv -> X.0zR
• com.instagram.mainactivity.MainActivity -> com.instagram.mainactivity.MainActivity
• com.instagram.ui.swipenavigation.SwipeNavigationContainer -> com.instagram.ui.swipenavigation.SwipeNavigationContainer
• X.1Fy -> X.1GD
• X.1J0 -> X.1Jo
• X.1JW -> X.1KI
• X.1Oe -> X.1Om
• com.instagram.feed.media.Media__JsonHelper -> com.instagram.feed.media.Media__JsonHelper
• X.1Pl -> X.1PZ
• X.1Qz -> X.1QB
• X.1gk -> X.1wn
• 

• X.1Pl#A03(X.0iA, X.0hu) -> X.1PZ#A03(X.0iT, X.0iD)

Matching Fields:
• LX/2vv;#A04 -> LX/39Y;#A04
• Lcom/instagram/direct/visual/DirectVisualMessageViewerController;#A0D -> Lcom/instagram/direct/visual/DirectVisualMessageViewerController;#A0D
• LX/3k8;#A01 -> LX/3FC;#A01
• LX/74b;#A07 -> LX/6Ay;#A07
• LX/4pb;#A04 -> LX/4lr;#A04
• LX/1Oe;#A0H -> LX/1Om;#A0H
• LX/0r4;#A00 -> LX/0ob;#A00
• LX/78j;#A0E -> LX/6x1;#A0E
• LX/1JW;#A0T -> LX/1KI;#A0R
• LX/1OL;#A06 -> LX/1OR;#A06
• LX/1k2;#A09 -> LX/1kr;#A09
• LX/1k2;#A08 -> LX/1kr;#A08
• LX/DJ0;#A00 -> LX/Amd;#A00
• LX/DJ1;#A00 -> LX/Ame;#A00
• LX/DJ0;#A01 -> LX/Amd;#A01
• LX/DJ1;#A01 -> LX/Ame;#A01
• LX/3Gz;#A09 -> LX/2se;#A09
• LX/32w;#A0D -> LX/34T;#A0D
• Lcom/instagram/reels/fragment/ReelViewerFragment;#A1G -> Lcom/instagram/reels/fragment/ReelViewerFragment;#A1I
• Lcom/instagram/reels/fragment/ReelViewerFragment;#mVideoPlayer -> Lcom/instagram/reels/fragment/ReelViewerFragment;#mVideoPlayer
• Lcom/instagram/reels/fragment/ReelViewerFragm

In [18]:
%autoreload

generate.generate_m_decs(m_decs, r_mas, accumulator.matching_ms)
generate.generate_c_decs(orig_c_decs, r_cas, accumulator.matching_cs)
generate.generate_f_decs(f_decs, r_fas, accumulator.matching_fs)

m_decs = (
    MethodDec('com.instagram.mainactivity.MainActivity', 'onStart'),
    MethodDec('X.39Y', 'onLongPress', 'android.view.MotionEvent'),
    MethodDec('com.instagram.debug.devoptions.api.DeveloperOptionsLauncher', 'loadAndLaunchDeveloperOptions', 'android.content.Context', 'X.1DA', 'androidx.fragment.app.FragmentActivity', 'X.0Ez', 'android.os.Bundle'),
    # No match for Method com.instagram.debug.devoptions.api.DeveloperOptionsLauncher#launchStoryToolbarSwitcherTool(android.content.Context, androidx.fragment.app.FragmentActivity, X.0C8),
    # No match for Method com.instagram.debug.devoptions.api.DeveloperOptionsLauncher#launchStoriesExperimentSwitcherTool(android.content.Context, androidx.fragment.app.FragmentActivity, X.0C8),
    MethodDec('com.instagram.direct.visual.DirectVisualMessageViewerController', 'A0C', 'com.instagram.direct.visual.DirectVisualMessageViewerController', 'java.lang.String'),
    MethodDec('com.instagram.direct.visual.DirectVisualMessageViewerContr

# Interoptability with Akrolyb

The following Code snippet allows to "extract" MethodDecs from MemberDeclarations in Akrolyb. This automates the process of providing GraphGuard the hooks it should find. Just paste the output (tuple of MethodDecs) in the corresponding section in this Notebook.

## Extract

It is not in Python, since it would require a Kotlin Parser and evaluating imports, variables etc. Just executing the Constructors in Kotlin, then getting the values is much easier than static analysis. 

The `main` function can (and should) be run statically (locally on the machine and not on your Android) to build the list of `MethodDec`s that GraphGuard is supposed to match in an updated build of the target application. Adjust the MemberDeclarations Class to the Class where you declared the `MemberDeclaration`s.

Note: This code does obviously not know if you left out the parameters on purpose (a lot of parameter types, or multiple constructors...). Please modify the results for the Methods concerned by using the optional `skip_params=True` argument for `MethodDec`.

```kotlin
fun main() {
    fun MemberDec.formatToGraphGuard() = buildString {
        append("MethodDec('")
        append(classDec.className)
        append("', '")
        append(if (this@formatToGraphGuard is MethodDec) methodName else "<init>")
        append("'")
        for (param in params) {
            append(", '")
            if (param is String)
                append(param)
            else if (param is ClassDec)
                append(param.className)
            else if (param is Class<*>)
                append(param.name)
            else error("Illegal Type for param")
            append("'")
        }
        append(")")
    }
    
    println("m_decs = (")
    val strs = MemberDeclarations::class.java.declaredFields.mapNotNull {
        it.isAccessible = true
        val x = it.get(MemberDeclarations)
        if (x !is MemberDec) return@mapNotNull null

        x.formatToGraphGuard()
    }.joinToString(separator = ",\n\t", prefix = "\t")
    println(strs)
    println(")")
}
```